# RestReqFactory

In [ ]:
#| default_exp RestReqFactory

In [ ]:
#| export
from dataclasses import dataclass
from typing import Union, Optional, Dict
from collections import defaultdict
import requests
import json

## `ENV` map to handle env variables

In [ ]:
#| export
ENV = defaultdict(lambda: None)
def get_env(
    x:str # key to filter  
): 
    return ENV[x] if ENV[x] else ''

In [ ]:
assert get_env('x') == ''

In [ ]:
#| export
def set_env(
    key:str,  # key to map variable  
    val:str   # value of the key
): 
    ENV[key] = val

In [ ]:
set_env('x', 3)
assert get_env('x') == 3

## RestReq
Base class to hold attributes of the rest request.

In [ ]:
#| export
allowed_methods = ["GET","POST","PUT", "DELETE"]

@dataclass
class RestReq:
    "Rest request building method"
    method: str                           # represents the request method
    url: str                              # URL 
    headers: Union[dict,None] = None      # headers
    params: Union[dict,None]= None        # parameters
    body: Union[dict, None] = None        # body
    kwargs: Union[dict, None] = None      # addtional args of `requests.request`
    
    @property
    def curl(self):
        "returing the curl repr of the rest request"
        params_str = "?"+"&".join([f"{key}={value}" for key, value in self.params.items()]) if self.params else ''
            
        curl_cmnd = f"curl -X {self.method.upper()} '{self.url}{params_str}'"

        if self.headers:
            for key, value in self.headers.items():
                curl_cmnd += f" -H '{key}: {value}'"
        
        if self.body :
            curl_cmnd += f" -d '{json.dumps(self.body)}'"
            
        return curl_cmnd

    def __call__(self, **kwargs):
        "invoke rest request"
        response = None
        try:
            if self.method.upper() in allowed_methods:
                response = requests.request(self.method, self.url, headers=self.headers, params=self.params, data=json.dumps(self.body), **kwargs)
            else:
                raise "not a valid method"
        
        except Exception as e:
            print(f"Error occurred while making the HTTP request: {e}")
        
        finally:
            return response

    def parse_kwargs(self):
        """
        process kwars set during the request, 
        for now it is converting cookies to dictionary/json parsable
        """
        if self.kwargs:
            for k,v in self.kwargs.items():
                if k == 'cookies' and isinstance(self.kwargs[k], requests.cookies.RequestsCookieJar ):
                    cookies = self.kwargs[k]
                    arr = []
                    for c in cookies:
                        arr.append({
                            'name': c.name,
                            'value': c.value,
                            'domain': c.domain,
                            'path': c.path
                        })
                    
                    self.kwargs[k] = arr

    def __str__(self):
        """
        print the request details as a JSON object.

        Returns:
            str: A JSON string representation of the request details.
        """
        self.parse_kwargs()
        req_details = {
            'method': self.method,
            'url': self.url,
            'headers': self.headers if self.headers else 'None',
            'params': self.params if self.params else 'None',
            'body': self.body if self.body else 'None',
            'kwargs': self.kwargs if self.kwargs else 'None'
        }
        
        return json.dumps(req_details, indent=4)

    __repr__ = __str__

In [ ]:

req = RestReq(
    method="POST",
    url='https://httpbin.org/post',
    headers={
    "accept": "application/json",
    },
)


### To extract curl repr

In [ ]:
req.curl

"curl -X POST 'https://httpbin.org/post' -H 'accept: application/json'"

### To invoke the rest call

In [ ]:
resp = req()
type(resp)

requests.models.Response

## RestReqFactory
Class to bind the lambda function to create `RestReq` object`

In [ ]:
#| export
class RestReqFactory:
    "Class to bind the lambda function to create RestReq object"
    def __init__(self, method:str,             # represents the request method
                 url_provider:lambda:None,     # function returning url 
                 headers_provider=lambda:None, # function returning headers 
                 params_provider=lambda:None,  # function returning params 
                 body_provider=lambda:None     # function returning params 
                ):
        self.method = method                      
        self.url_provider = url_provider
        self.headers_provider = headers_provider
        self.params_provider = params_provider
        self.body_provider = body_provider
        self.kwargs = None
    
    def __call__(self):
        return RestReq(
            method=self.method,
            url=self.url_provider(),
            headers=self.headers_provider(),
            params=self.params_provider(),
            body=self.body_provider(),
            kwargs=self.kwargs 
        )
        
    def set_req_kwargs(self, kwargs):
        " set `RestReq` object with the `kwargs`"
        self.kwargs = kwargs

### Build basic `RestReqFactory` object

In [ ]:
url = lambda: f"{get_env('url')}/post"

head = lambda : {
    "accept": f"{get_env('accept')}",
}

# Create the RestReqFactory instance
req = RestReqFactory(
    method="POST",
    url_provider=url,
    headers_provider=head,
)

In [ ]:
assert req().curl == """curl -X POST '/post' -H 'accept: '"""

### leveraging Dynammic nature of `ENV` 

In [ ]:
set_env('url', 'https://httpbin.org')
set_env('accept', 'application/json')

In [ ]:
assert req().curl == """curl -X POST 'https://httpbin.org/post' -H 'accept: application/json'"""

In [ ]:
resp = req()()
assert resp.status_code  == 200

In [ ]:
req()

{
    "method": "POST",
    "url": "https://httpbin.org/post",
    "headers": {
        "accept": "application/json"
    },
    "params": "None",
    "body": "None",
    "kwargs": "None"
}

### Cookies testing

In [ ]:
url = lambda: f"{get_env('postman-url')}/get"

params = lambda : {
    "foo1": f"{get_env('foo1')}",
    "foo2": f"{get_env('foo2')}",
}

# Create the RestReqFactory instance
req = RestReqFactory(
    method="GET",
    url_provider=url,
    params_provider=params,
)

In [ ]:
set_env('foo1', 'foo1')
set_env('foo2', 'foo2')
set_env('postman-url', 'https://postman-echo.com')

Setting up the kwargs field for the req object 

In [ ]:
res = req()()
assert res.status_code == 200
req.set_req_kwargs({'cookies' : res.cookies})

check `__str__` and `__repr__`

In [ ]:
req()

{
    "method": "GET",
    "url": "https://postman-echo.com/get",
    "headers": "None",
    "params": {
        "foo1": "foo1",
        "foo2": "foo2"
    },
    "body": "None",
    "kwargs": {
        "cookies": [
            {
                "name": "sails.sid",
                "value": "s%3Af2Ma6_qtUOO6o8rLJuS0_oGC3nf42AGn.4f5n7pa3spk6owtt0P1lt4lsELJVx%2FTZUwJ7w97sb6Y",
                "domain": "postman-echo.com",
                "path": "/"
            }
        ]
    }
}

In [ ]:
#| hide
import nbdev

In [ ]:
#| hide
nbdev.export.nb_export('RestReqFactory.ipynb','../FastRequest/')